### This is the notebook for lyrics classification using BERT.

* Author: Alperen Demirci, Bora Dere

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
## import transformers library
import torch
from transformers import BertTokenizer, BertModel
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split


In [4]:
data = pd.read_csv("../data/cleaned_sampled_disorder_musics.csv")

label_dict = {
    'depression' : 0,
    'anxiety' : 1,
    'bipolar' : 2,
    'ptsd' : 3,
    'borderline' : 4,
    'panic' : 5,

}

labels = data['disorder'].map(label_dict)

In [5]:
data

,artist,lyric,type,title,user_id,src,disorder,lyric_length,unique_user_count
0,echo & the bunnymen,[verse 1]\nunder blue moon i saw you\nso soon ...,P,the killing moon,4353e884c1,SPOTIFY,anxiety,1159,436
1,louis tomlinson,[verse 1: bebe rexha]\ni know you say you know...,P,back to you (feat. bebe rexha & digital farm a...,b170e4337a,SPOTIFY,anxiety,2071,436
2,adele,[verse 1]\ndaydreamer\nsitting on the sea\nsoa...,P,daydreamer,57b5175168,SPOTIFY,anxiety,1008,436
3,one direction,"[verse 1: liam]\nyou're insecure, don't know w...",P,what makes you beautiful,8bdda90e71,SPOTIFY,anxiety,2308,436
4,beach weather,[verse 1: nick santino]\nlate night telephone\...,P,"sex, drugs, etc.",dd044ed3a8,SPOTIFY,anxiety,1242,436
...,...,...,...,...,...,...,...,...,...
3267,gorillaz,[hook: 2-d & brandon markell holmes]\ncircle o...,A,circle of friendz (feat. brandon markell holmes),4d1356518e,APPLE,ptsd,841,245
3268,the mighty mighty bosstones,[verse 1]\nwell he was fueled by a lack\ndrew ...,P,the rascal king,1bf579bc57,SPOTIFY,ptsd,1400,245
3269,nocap,"[pre-chorus]\nyeah, didn't write this song, bu...",A,ghetto angels,e77b72eef7,APPLE,ptsd,2461,245
3270,ynw melly,"[intro]\nyeah, yeah, yeah\nyoung nigga world\n...",A,virtual (blue balenciagas),7b53df78b5,APPLE,ptsd,2904,245


In [10]:
# Train-test split
train_texts, val_texts, train_labels, val_labels = train_test_split(
    data["lyric"], labels, test_size=0.2, random_state=42, stratify=labels
)

In [11]:
train_df = pd.DataFrame({
    'lyric': train_texts,
    'label': train_labels
})

val_df = pd.DataFrame({
    'lyric': val_texts,
    'label': val_labels
})

train_df.to_csv("../data/train_musics.csv", index=False)
val_df.to_csv("../data/val_musics.csv", index=False)

In [13]:
train_features, val_features, train_labels, val_labels = train_test_split(
    data.drop('lyric', axis=1), labels, test_size=0.2, random_state=42, stratify=labels
)

In [14]:
train_features

,artist,type,title,user_id,src,disorder,lyric_length,unique_user_count
2281,yg,T,toot it and boot it,7f20349dc1,SPOTIFY,panic,2584,23
1249,hailee steinfeld,P,love myself,3324437f73,SPOTIFY,borderline,2408,48
802,super furry animals,P,zoom!,8222eada89,SPOTIFY,bipolar,957,133
317,the maine,A,slip the noose (orchestral),d9d9416fb0,SPOTIFY,anxiety,1794,436
1364,girl in red,P,girls,48e275ed8e,SPOTIFY,borderline,1178,48
...,...,...,...,...,...,...,...,...
2072,the killers,T,mr. brightside,d066bd53a8,SPOTIFY,depression,1546,772
412,andrew bird,A,andalucia,6b17239a5b,SPOTIFY,anxiety,647,436
3237,neil diamond,A,"cherry, cherry",07a651a06f,APPLE,ptsd,1160,245
834,solange,A,"interlude: i got so much magic, you can have i...",94a96931df,APPLE,bipolar,307,133


In [16]:
train_features['disorder'] = train_labels
val_features['disorder'] = val_labels

train_features.to_csv("../data/train_musics_features.csv", index=False)
val_features.to_csv("../data/val_musics_features.csv", index=False)

In [17]:
train_df = pd.read_csv("../data/train_musics.csv")

In [18]:
train_df

,lyric,label
0,"[hook: ty dolla sign]\ni met her at the club, ...",5
1,"[written by julia michaels, justin tranter, os...",4
2,took you to a movie but you moved away\nfroze ...,2
3,"[chorus]\nbreak down, hysteric and young\nunco...",1
4,[verse 1]\ni've been hiding for so long\nthese...,4
...,...,...
2612,[verse 1]\ncoming out of my cage and i've been...,0
2613,[verse 1]\nandalucia\nwhen can i see you\nwhen...,1
2614,"baby loves me, yes, yes she does\nah, the girl...",3
2615,[a cappella interlude: nia andrews and kelly r...,2


In [25]:
val_df

,text,label
56586,WAIT SO IF A DIFFERENT ACC HAS IT U CAN USE I...,3
39388,i &lt;3 spike #AnimalCrossing #ACNH #NintendoS...,0
46164,Got way too drunk last night and ruined my lif...,5
27178,I just voted for to win MV Congeniality Star ...,4
42087,What I'm trying to say is we're super casual a...,5
...,...,...
27502,I want to customize my car horn to play the An...,4
45085,Just finished watching and I NEED SEASON 4 #S...,5
675,I used to have a hourglass shaped body. I used...,1
59713,We will miss you. I remember that unforgett...,3


### load the cleaned text data

In [3]:
train_df = pd.read_csv("../data/processed_train_musics.csv")
val_df = pd.read_csv("../data/processed_val_musics.csv")

In [4]:
val_df.dropna(inplace=True)
train_df.dropna(inplace=True)

train_df.drop('Unnamed: 0', axis=1, inplace=True)
val_df.drop('Unnamed: 0', axis=1, inplace=True)

In [5]:
train_df

,lyric,label
0,"[ hook : ty dolla sign ] meet club , say `` wa...",5
1,"[ write julia michael , justin tranter , oscar...",4
2,"take movie move away froze standstill , foot t...",2
3,"[ chorus ] break down , hysteric young uncomfo...",1
4,"[ verse 1 ] i 've hide long feeling , they 're...",4
...,...,...
2612,"[ verse 1 ] come cage i 've fine get ta , get ...",0
2613,[ verse 1 ] andalucia see snow farmer john wan...,1
2614,"baby love me , yes , yes ah , girl 's outta si...",3
2615,[ a cappella interlude : nia andrew kelly rowl...,2


In [6]:
from transformers import BertTokenizer, BertModel
from transformers import AutoTokenizer, AutoModel

import torch
from tqdm import tqdm
import pandas as pd

data = train_df

model_name = 'brunokreiner/lyrics-bert'

# Load model from HuggingFace Hub
tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')
model = AutoModel.from_pretrained('bert-base-uncased')

#Mean Pooling - Take attention mask into account for correct averaging
def mean_pooling(model_output, attention_mask):
    token_embeddings = model_output[0] #First element of model_output contains all token embeddings
    input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
    return torch.sum(token_embeddings * input_mask_expanded, 1) / torch.clamp(input_mask_expanded.sum(1), min=1e-9)


data['lyric'] = data['lyric'].astype(str)

sentences = data.lyric.values  # Ensure this is a list of strings
encoded_input = tokenizer(list(sentences), padding=True, truncation=True, return_tensors='pt')

# Compute token embeddings
with torch.no_grad():
    model_output = model(**encoded_input)

# Perform pooling. In this case, mean pooling.
sentence_embeddings = mean_pooling(model_output, encoded_input['attention_mask'])


# Convert features to a DataFrame and save them
features_df = pd.DataFrame(sentence_embeddings)
features_df['label'] = data['label']  # Add the original labels
features_df.to_csv("bert_musicfeatures_train.csv", index=False)

print("Feature extraction complete! Saved to bert_features.csv.")


KeyboardInterrupt: 

: 

In [32]:
%pip install -U sentence-transformers

Defaulting to user installation because normal site-packages is not writeable
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.1/10.1 MB 1.9 MB/s eta 0:00:0000:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 2.1 MB/s eta 0:00:00a 0:00:01

[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: pip3 install --upgrade pip
Note: you may need to restart the kernel to use updated packages.
